In [ ]:
# Install required packages
!pip install transformers
!pip install datasets
!pip install sentencepiece

In [ ]:
# load created datasets (This is for JParaCrawl version 3.0)
from datasets import load_from_disk

pre = "drive/MyDrive/Language_translation/data/"
filename_1 = "en-jp-v3.0.1"
filename_2 = "en-jp-v3.0.2"
filename_3 = "en-jp-v3.0.3"
filename_4 = "en-jp-v3.0.4"
filename_5 = "en-jp-v3.0.5"
dataset_1 = load_from_disk(pre + filename_1)
dataset_2 = load_from_disk(pre + filename_2)
dataset_3 = load_from_disk(pre + filename_3)
dataset_4 = load_from_disk(pre + filename_4)
dataset_5 = load_from_disk(pre + filename_5)

In [ ]:
# This step is only for JParaCrawl version 3.0
# concatenate all the datasets
from datasets import concatenate_datasets

# datasets lists
datasets_list = [dataset_1, dataset_2, dataset_3, dataset_4, dataset_5]

# concatenate the datasets
combined_dataset = concatenate_datasets(datasets_list)

In [ ]:
# clean for final dataset
# function to split data 80% train, 10% validation, 10% test
def create_splits(data):
  first_split = data.train_test_split(test_size=0.20)
  train = first_split.pop("train")
  second_split = first_split["test"].train_test_split(test_size=0.50)
  validation = second_split.pop("train")
  test = second_split.pop("test")
  second_split["train"] = train
  second_split["validation"] = validation
  second_split["test"] = test
  return second_split

In [ ]:
# filter_data
data = combined_dataset.filter(lambda x: float(x['score']) > 0.77)

# shuffle data
data = data.shuffle()

# remove source and score columns
data = data.remove_columns(["source", "score"])

# split the dataset into train, validation and test datasetdict
data_split_1 = create_splits(data)

In [ ]:
 # nested function to create a translation column for datasetdict
def create_translation_column(example):
  example["translation"] = {'en': example["en"], 'jp': example["jp"]}
  return example

# create the translation
data_with_trans = data_split_1.map(create_translation_column)

In [ ]:
# remove the 'en' and 'jp' columns
final_data = data_with_trans.remove_columns(["en", "jp"])

# save to disk
final_data.save_to_disk("drive/MyDrive/Language_tranlsation/data/en-jp-v3.0-subset-sc-ov-77")